<a href="https://colab.research.google.com/github/Pedrohero13/DB_NO_CON_colab/blob/main/peliculas_netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de la librería streamlit

In [1]:
!pip install streamlit


Vamos a cargar una máquina virtual que tenga instalado Ngrok

In [37]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-02-21 18:21:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  6.59MB/s    in 2.0s    

2022-02-21 18:21:45 (6.59 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]



Ahora cargaremos esta máquina virtual en nuestro archivo de Google Colab

In [113]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Add token ngrok to ngrok suite

In [142]:
get_ipython().system_raw('./ngrok authtoken 24qeTD0luOCmIfQYHBvCGHHddbz_5CF1HTN82FtBvpFrd8yhf')

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Posteriormente le indicaremos a Ngrok que estará utilizando el puerto 8501 para visualizar nuestra aplicación

In [143]:
get_ipython().system_raw('./ngrok http 8501 &')

Una vez inicializado el puerto, ahora vamos a crear el tunel para que pueda generarse un enlace que nos permita visualizar nuestra aplicación

In [144]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://9f4c-35-221-215-102.ngrok.io


Una vez que ejecutaron la instrucción anterior, verán que aparece un URL que es el que vamos a dar clic para visualizar nuestra aplicación, pero primero tenemos que ejecutar nuestra aplicación

**Práctica 1**
**Caso Netflix : Dashboard para visualizar datos de filmes**

Ahora creamos nuestra aplicacion, para guardar el archivo nos aseguramos de incluir la sentencia: %%writefile netflix.py

In [141]:
%%writefile movies.py
import pandas as pd
import numpy as np
import streamlit as st
import codecs
import re
st.title('Peliculas de Netflix')


DATA_URL = ('/content/movies.csv')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, encoding_errors='ignore')
    return data

#Cargar datos si se selecciona en el checkbox
agree = st.sidebar.checkbox("¿Quieres mostrar todos los datos? ")
if agree:
  data_load_state = st.text('Cargando...')
  data = load_data(2000)
  data_load_state.text("Cargado! (using st.cache)")
  st.dataframe(data)



#filtlar por el nombre de la pelicula
@st.cache
def load_data_byname(name):
  datafiltered = load_data(2000)
  filtered_data_byname = datafiltered[datafiltered['name'].str.contains(name,flags=re.IGNORECASE)]
  
  return filtered_data_byname

titulo = st.sidebar.text_input('Titulo de la pelicula: ')
btnFilteredMovie = st.sidebar.button('Buscar Pelicula')

if (btnFilteredMovie):
  st.write ("Titulo buscado: "+ titulo)
  filterbyname = load_data_byname(titulo)
  count_row = filterbyname.shape[0]
  st.write(f'Total de peliculas: {count_row}')

  st.dataframe(filterbyname)


#filtrar por el nombre del direcctor 
@st.cache
def load_data_bydire(director):
  data = load_data(2000)
  filtered_data_bydire = data[data['director'] == director]
  return filtered_data_bydire

data = load_data(2000)
selected = st.sidebar.selectbox("Selecciona el director", data['director'].unique())
btnFilterBydire = st.sidebar.button('Filtrar por Director')

if (btnFilterBydire): 
  st.write("Peliculas dirigidas por "+selected)
  filterbydire = load_data_bydire(selected)
  count_row = filterbydire.shape[0]
  st.write(f'Total Peliculas: {count_row}')

  st.dataframe(filterbydire)


Overwriting movies.py


Una vez cargado y ejecutado el archivo ahora vamos a proceder a ejecutarlo con el comando streamlit para que se visualice el contenido del mismo.

In [ ]:
!streamlit run /content/movies.py

2022-02-21 19:21:16.522 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.215.102:8501

